In [1]:
!pip uninstall transformers --yes
!pip install unsloth
!pip install bitsandbytes

Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 3.1 MB/s eta 0:00:00
 

In [2]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, DataCollatorWithPadding, DataCollatorForSeq2Seq

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
from datasets import load_dataset
from pathlib import Path

files = list(map(str, Path("/kaggle/input/wiki-20220301-en-sci").glob("*.parquet")))
dataset = load_dataset("parquet", data_files=files, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset[:3]

{'text': ['The Ulakhan Fault is a left-lateral moving transform fault that runs along the boundary between two tectonic plates in northeast Asia, the North American Plate, and the Okhotsk Plate.  It runs from a triple junction in the Chersky Range in the west, to another triple junction with the Aleutian Trench and the Kuril Trench in the east. From the offset of dated geomorphological features, a slip rate of 5.3±1.3 mm per year has been measured, consistent with estimates from GPS constrained global plate models. The analysis of fault scarps along the fault zone in the Seymchan Basin suggests that the fault is characterised by occasional large (>7.5) earthquakes.\n\nReferences\n\nExternal links\n International Seismic-Volcanic Workshop on Kamchatkan-Aleutian Subduction Processes (KASP), Fourth Workshop, Petropavlovsk-Kamchatsky August 21-27, 2004\n\nPlate tectonics\nSeismic faults of Asia\nGeology of the Russian Far East\nSeismic faults of North America',
  'Tripodal ligands are tri-

In [5]:
from unsloth import FastModel

<ipython-input-5-df41f2f601f1>:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
FastModel.from_pretrained?

In [7]:
model, tokenizer = FastModel.from_pretrained(
        model_name = "unsloth/Llama-3.2-1B",
        max_seq_length = 512, 
        dtype = torch.bfloat16,
        full_finetuning = True,
        load_in_4bit = False,
        load_in_8bit = False,
        device_map = 'auto'
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    NVIDIA L4. Num GPUs = 4. Max memory: 22.278 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [8]:
from trl import SFTTrainer,SFTConfig

training_args = SFTConfig(
    warmup_ratio=0.1, 
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    # per_device_eval_batch_size=2,
    num_train_epochs=1,
    report_to='none',
    output_dir = f'./checkpoints',
    # fp16=True,
    bf16=True,
    optim='paged_adamw_32bit', #'paged_adamw_32bit', 'paged_adamw_8bit', 'adamw_torch'
    torch_compile=True, #model = torch.compile(model)
    gradient_accumulation_steps=1,
    logging_steps=100,
    save_strategy="steps",
    save_only_model=True,
    save_total_limit=1,
    save_steps=100,
    lr_scheduler_type='cosine',
    weight_decay=0.01,
)

In [9]:
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    dataset_text_field='text',
    train_dataset=dataset,
    args=training_args,
)

trainer.train()
trainer.save_model(f'./Llama3.2-1b-wiki')

Unsloth: Tokenizing ["text"] (num_proc=48):   0%|          | 0/131049 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 4
   \\   /|    Num examples = 131,049 | Num Epochs = 1 | Total steps = 4,096
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 1,235,814,400/1,235,814,400 (100.00% trained)


Unsloth: Enabled auto compiling


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
100,2.214300
200,2.149500
300,2.129700
400,2.124200
500,2.107600
600,2.104200
700,2.108800
800,2.103000
900,2.086900
1000,2.101100


Unsloth: Will smartly offload gradients to save VRAM!
